## Project 5: Vehicle Detection
### Author: Tri Minh Cao
### Email: trimcao@gmail.com
### Date: July 2017

#### Tasks
- Obtain HOG and color features for the training data. 
    - Need to choose color and channel that work best (need to experiment)
    - Split data into training and validation set
- Train a SVM classifier
    - Use RandomSearch to find the best parameters for the classifier.
    - For other parameters (related to features), need to do my own experiments.
- Sliding window
    - Multi-scale window (various sizes for each part of the road)
- HOG sub-sampling window search
- Heat map to avoid false positives
    - Find the centroid of multiple windows (in case of multiple true positives).

In [22]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split
import os
%matplotlib inline
plt.rcParams["figure.figsize"] = [12,7]

### Lesson Functions

In [2]:
def color_hist(img, nbins=32, bins_range=(0,256)):
    """
    Compute the histogram of color channels separately.
    # NEED TO CHANGE bins_range if reading .png files with mpimg!
    """
    hist0 = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    hist1 = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    hist2 = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate to create a feature vector
    hist_features = np.concatenate((hist0[0], hist1[0], hist2[0]))
    return hist_features

In [3]:
def bin_spatial(img, size=(32,32)):
    """
    Compute binned color features. 
    """
    small_img = cv2.resize(img, size)
    features = small_img.ravel()
    return features

In [4]:
def get_hog_features(img, orient, pix_per_cell, cell_per_block,
                     vis=False, feature_vec=True):
    """
    Get HOG features from an image.
    """
    if vis:
        features, hog_image = hog(img, orientations=orient,
                                  pixels_per_cell=(pix_per_cell,pix_per_cell),
                                  cells_per_block=(cell_per_block,cell_per_block),
                                  transform_sqrt=True,
                                  visualise=vis, feature_vector=feature_vec)   
        return features, hog_image
    else:
        features = hog(img, orientations=orient,
                       pixels_per_cell=(pix_per_cell,pix_per_cell),
                       cells_per_block=(cell_per_block,cell_per_block),
                       transform_sqrt=True,
                       visualise=vis, feature_vector=feature_vec)
        return features

In [5]:
def extract_features(imgs, color_space='RGB', spatial_size=(32,32),
                     hist_bins=32, orient=9, 
                     pix_per_cell=8, cell_per_block=2, hog_channel=0,
                     spatial_feat=True, hist_feat=True, hog_feat=True):
    """
    Extract spatial, color histogram, and HOG features from images.
    """
    # Create a list of feature vectors
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # image = mpimg.imread(file)
        image = cv2.imread(file) 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # apply color conversion if color_space is not RGB
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)  
        # start obtaining features
        if spatial_feat:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat:
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat:
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel],
                                                         orient, pix_per_cell, cell_per_block,
                                                         vis=False, feature_vec=True))
                # ravel the feature list
                hog_features = np.ravel(hog_features)
                # for each in hog_features:
                #    print(each)
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel],
                                                orient, pix_per_cell, cell_per_block,
                                                vis=False, feature_vec=True)
            file_features.append(hog_features)
        # Concatenate all extracted features of a single image    
        features.append(np.concatenate(file_features))
    return features

In [6]:
def draw_boxes(img, bboxes, color=(0,0,255), thick=6):
    """
    Draw bounding boxes.
    """
    imcopy = np.copy(img)
    for bbox in bboxes:
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    return imcopy
    

In [7]:
def slide_window(img, x_start_stop=[None,None], y_start_stop=[None,None],
                 xy_window=(64,64), xy_overlap=(0.5,0.5)):
    """
    Sliding a window and obtain an image for each window.
    """
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

In [8]:
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)


### Obtain image data

In [9]:
os.listdir('./vehicles/')

['.DS_Store',
 'GTI_Far',
 'GTI_Left',
 'GTI_MiddleClose',
 'GTI_Right',
 'KITTI_extracted']

In [10]:
def get_image_path(path):
    """
    Get all images path from a folder.
    Note that the actual images reside in *one* more folder layer.
    """
    paths = os.listdir(path)
    images = []
    for each in paths:
        cur_path = path + each + '/'
        images.extend(glob.glob(cur_path + '*.png'))
    return images

In [11]:
# Get the list of image paths
cars = get_image_path('./vehicles/')
notcars = get_image_path('./non-vehicles/')

In [12]:
print(cars[0])

./vehicles/GTI_Far/image0000.png


### Train a classifier

In [79]:
# parameters
color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 6  # HOG orientations (before 9)
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16) # Spatial binning dimensions (before 32, 32)
hist_bins = 32    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off

y_start_stop = [400,700] # Min and max in y to search in slide_window()

In [80]:
# Extracting features
car_features = extract_features(cars, color_space, spatial_size, hist_bins,
                                orient, pix_per_cell, cell_per_block,
                                hog_channel, spatial_feat, hist_feat,
                                hog_feat)
notcar_features = extract_features(notcars, color_space, spatial_size, hist_bins,
                                   orient, pix_per_cell, cell_per_block,
                                   hog_channel, spatial_feat, hist_feat,
                                   hog_feat)

In [81]:
print(len(car_features[0]))

4392


In [82]:
# Create train and test sets  

X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

In [83]:
# Train a classifier

# Use a linear SVC 
svc = LinearSVC()
# svc = SVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))

11.98 Seconds to train SVC...
Test Accuracy of SVC =  0.991


In [84]:
# Save the classifier 
import pickle
filename = 'svm_linear_071417.pkl'
pickle.dump(svc, open(filename, 'wb'))

### Sliding window search

In [ ]:
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows

In [ ]:
image = cv2.imread('./test_images/test6.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
draw_image = np.copy(image)

windows = slide_window(image, x_start_stop=[None, None], y_start_stop=y_start_stop, 
                    xy_window=(96, 96), xy_overlap=(0.5, 0.5))

hot_windows = search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)                       

window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)   

plt.rcParams["figure.figsize"] = [12,7]
plt.imshow(window_img)


In [ ]:
# draw the test windows (not just the hot ones)
reg_windows = slide_window(image, x_start_stop=[None, None], y_start_stop=[400, 650], 
                    xy_window=(96, 96), xy_overlap=(0.5, 0.5))
reg_window_img = draw_boxes(draw_image, reg_windows, color=(0, 0, 255), thick=6)   
plt.imshow(reg_window_img)

In [ ]:
bigger_windows = slide_window(image, x_start_stop=[None, None], y_start_stop=[350, 720], 
                    xy_window=(240, 240), xy_overlap=(0.75, 0.75))
bigger_window_img = draw_boxes(draw_image,  bigger_windows, color=(0, 0, 255), thick=6)   
plt.imshow(bigger_window_img)

In [ ]:
big_windows = slide_window(image, x_start_stop=[None, None], y_start_stop=[360, 600], 
                    xy_window=(150, 150), xy_overlap=(0.75, 0.75))
big_window_img = draw_boxes(draw_image,  big_windows, color=(0, 0, 255), thick=6)   
plt.imshow(big_window_img)

In [ ]:
small_windows = slide_window(image, x_start_stop=[None, None], y_start_stop=[380, 540], 
                    xy_window=(48, 48), xy_overlap=(0.5, 0.5))
small_window_img = draw_boxes(draw_image,  small_windows, color=(0, 0, 255), thick=6)   
plt.imshow(small_window_img)

In [ ]:
# Set the parametersv for sliding windows
xy_overlap = [0.5,0.5]

# x_start_stop_small = [400, 1000]
x_start_stop_small = [None, None]
y_start_stop_small = [350, 540]
xy_window_small = [48, 48]

x_start_stop_reg = [None, None]
y_start_stop_reg = [400, 660]
xy_window_reg = [96,96]

x_start_stop_big = [None, None]
y_start_stop_big = [360, 630]
xy_window_big = [150,150]
xy_overlap_big = [0.75, 0.75]

x_start_stop_bigger = [None, None]
y_start_stop_bigger = [350, 720]
xy_window_bigger = [240,240]
xy_overlap_bigger = [0.75, 0.75]

In [ ]:
image = cv2.imread('./test_images/test6.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
draw_image = np.copy(image)

windows = []
# add small windows
windows.extend(slide_window(image, x_start_stop=x_start_stop_small, y_start_stop=y_start_stop_small, 
                    xy_window=xy_window_small, xy_overlap=xy_overlap))
# add reg windows
windows.extend(slide_window(image, x_start_stop=x_start_stop_reg, y_start_stop=y_start_stop_reg, 
                    xy_window=xy_window_reg, xy_overlap=xy_overlap))
# add big windows
windows.extend(slide_window(image, x_start_stop=x_start_stop_big, y_start_stop=y_start_stop_big, 
                    xy_window=xy_window_big, xy_overlap=xy_overlap_big))
# add bigger windows
windows.extend(slide_window(image, x_start_stop=x_start_stop_bigger, y_start_stop=y_start_stop_bigger, 
                    xy_window=xy_window_bigger, xy_overlap=xy_overlap_bigger))

# windows.extend(slide_window(image, x_start_stop=[None,None], y_start_stop=[350,720], 
#                     xy_window=xy_window_reg, xy_overlap=xy_overlap))

hot_windows = search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)                       

window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)   

plt.rcParams["figure.figsize"] = [12,7]
plt.imshow(window_img)



#### Heatmap

In [63]:
from scipy.ndimage.measurements import label

# box_list = hot_windows
# heat = np.zeros_like(image[:,:,0]).astype(np.float)

def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

# # Add heat to each box in box list
# heat = add_heat(heat,box_list)
    
# # Apply threshold to help remove false positives
# threshold = 1
# heat = apply_threshold(heat,threshold)

# # Visualize the heatmap when displaying    
# heatmap = np.clip(heat, 0, 255)

# # Find final boxes from heatmap using label function
# labels = label(heatmap)
# draw_img = draw_labeled_bboxes(np.copy(image), labels)

# fig = plt.figure()
# plt.subplot(121)
# plt.imshow(draw_img)
# plt.title('Car Positions')
# plt.subplot(122)
# plt.imshow(heatmap, cmap='hot')
# plt.title('Heat Map')
# fig.tight_layout()

In [ ]:
print(len(labels))

In [ ]:
def find_cars(image):
    # image = cv2.imread(img_file)
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # draw_image = np.copy(image)

    windows = []
    # add small windows
    windows.extend(slide_window(image, x_start_stop=x_start_stop_small, y_start_stop=y_start_stop_small, 
                        xy_window=xy_window_small, xy_overlap=xy_overlap))
    # add reg windows
    windows.extend(slide_window(image, x_start_stop=x_start_stop_reg, y_start_stop=y_start_stop_reg, 
                        xy_window=xy_window_reg, xy_overlap=xy_overlap))
    # add big windows
    windows.extend(slide_window(image, x_start_stop=x_start_stop_big, y_start_stop=y_start_stop_big, 
                        xy_window=xy_window_big, xy_overlap=xy_overlap_big))
    # add bigger windows
    windows.extend(slide_window(image, x_start_stop=x_start_stop_bigger, y_start_stop=y_start_stop_bigger, 
                        xy_window=xy_window_bigger, xy_overlap=xy_overlap_bigger))

    # windows.extend(slide_window(image, x_start_stop=[None,None], y_start_stop=[350,720], 
    #                     xy_window=xy_window_reg, xy_overlap=xy_overlap))

    hot_windows = search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)                       
    # threshold
    box_list = hot_windows
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    # Add heat to each box in box list
    heat = add_heat(heat,box_list)

    # Apply threshold to help remove false positives
    threshold = 1
    heat = apply_threshold(heat,threshold)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(np.copy(image), labels)
    return draw_img

### HOG Sub-sampling

In [93]:
img = cv2.imread('./test_images/test4.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
scales = [1., 1.5, 2.]

# Define a single function that can extract features using hog sub-sampling and make predictions
def find_cars(img, ystart=400, ystop=650, scales=scales, svc=svc, X_scaler=X_scaler,
              orient=orient, pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
              spatial_size=spatial_size, hist_bins=hist_bins): 
    draw_img = np.copy(img)
    # img = img.astype(np.float32)/255
    
    img_tosearch = img[ystart:ystop,:,:]
    ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2YCrCb)
    # save the hot windows
    box_list = []
    for scale in scales:
        # if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))

        ch1 = ctrans_tosearch[:,:,0]
        ch2 = ctrans_tosearch[:,:,1]
        ch3 = ctrans_tosearch[:,:,2]

        # Define blocks and steps as above
        nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
        nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1 
        nfeat_per_block = orient*cell_per_block**2

        # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
        window = 64
        nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
        cells_per_step = 2  # Instead of overlap, define how many cells to step
        nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
        nysteps = (nyblocks - nblocks_per_window) // cells_per_step

        # Compute individual channel HOG features for the entire image
        hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
        hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
        hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)

        for xb in range(nxsteps):
            for yb in range(nysteps):
                ypos = yb*cells_per_step
                xpos = xb*cells_per_step
                # Extract HOG for this patch
                hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

                xleft = xpos*pix_per_cell
                ytop = ypos*pix_per_cell

                # Extract the image patch
                subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))

                # Get color features
                spatial_features = bin_spatial(subimg, size=spatial_size)
                hist_features = color_hist(subimg, nbins=hist_bins)

                # Scale features and make a prediction
                test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))    
                #test_features = X_scaler.transform(np.hstack((shape_feat, hist_feat)).reshape(1, -1))    
                test_prediction = svc.predict(test_features)

                if test_prediction == 1:
                    xbox_left = np.int(xleft*scale)
                    ytop_draw = np.int(ytop*scale)
                    win_draw = np.int(window*scale)
                    # cv2.rectangle(draw_img,(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart),(0,0,255),6)
                    # we got a hot window
                    box_list.append(((xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart)))
                
    
    # threshold
    # box_list = hot_windows
    heat = np.zeros_like(img[:,:,0]).astype(np.float)
    # Add heat to each box in box list
    heat = add_heat(heat,box_list)

    # Apply threshold to help remove false positives
    threshold = 1
    heat = apply_threshold(heat,threshold)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(np.copy(img), labels)
    return draw_img
    
ystart = 400
ystop = 656
# scale = 2
    
# out_img = find_cars(img, ystart, ystop, scales, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)

# plt.imshow(out_img)

In [91]:
# More parameters
ystart = 400
ystop = 656
scales = [1., 1.5, 2.]

In [62]:
def pipeline(image):
    # call find_cars() method
    # find heat map
    # output the final result
    out_img = find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    pass

In [ ]:
test_image1 = find_cars('./test_images/test6.jpg')
plt.imshow(test_image1)

In [95]:
# Quick test on a video
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
# Set the video output and input
output = './output/project5_test.mp4'
# output = 'output/project4_text_short.mp4'
# origin_clip = VideoFileClip('./project_video.mp4').subclip(18,30)
origin_clip = VideoFileClip('./project_video.mp4')
drawn_clip = origin_clip.fl_image(find_cars)
%time drawn_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video ./output/project5_test.mp4
[MoviePy] Writing video ./output/project5_test.mp4



 12%|█▏        | 152/1261 [03:36<26:16,  1.42s/it]


 24%|██▍       | 303/1261 [07:11<24:26,  1.53s/it]


 36%|███▌      | 454/1261 [11:05<24:36,  1.83s/it]


 48%|████▊     | 605/1261 [18:09<35:11,  3.22s/it]


...

 72%|███████▏  | 907/1261 [38:44<24:45,  4.20s/it]


 84%|████████▍ | 1057/1261 [53:03<17:51,  5.25s/it]


 96%|█████████▌| 1205/1261 [59:19<01:18,  1.40s/it]


100%|█████████▉| 1260/1261 [1:00:36<00:01,  1.40s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output/project5_test.mp4 

CPU times: user 37min 18s, sys: 1min 36s, total: 38min 55s
Wall time: 1h 38s
